In [2]:
import sys
sys.path.append("..")
import mlflow
import pandas as pd
import config.config as config
import tasks.datapipeline.process_data as process_data 

mlflow.set_tracking_uri('../mlruns')

In [3]:
#add noise to lor_er and age (for demo purposes)

import tasks.datapipeline.process_data as process_data 
import numpy as np 

conf = None
conf = config.read_config('../config/gemini.cfg')
data, _ = process_data.pipeline(conf)

print(data['age'].head)

mu, sigma = 5, 3 
# creating a noise with the same dimension as the dataset  
noise = np.random.normal(mu, sigma, len(data['age'])).astype(int) 
print(noise)

data['age'] = data['age'].add(noise)
print(data['age'].head)

#repeat for los_er
mu, sigma = 5, 5 
# creating a noise with the same dimension as the dataset  
noise = np.random.normal(mu, sigma, len(data['los_er'])).astype(int) 
print(noise)

data['los_er'] = data['los_er'].add(noise)
data.to_csv('/mnt/nfs/project/delirium/data/all_before_2018_with_noise.csv')

conf.input = '/mnt/nfs/project/delirium/data/all_before_2018_with_noise.csv'
data, _ = process_data.pipeline(conf)
print(data['age'].head)

<bound method NDFrame.head of 0         87
1         52
2         59
3         62
4         52
          ..
281560    54
281561    85
281562    53
281563    80
281564    86
Name: age, Length: 281565, dtype: int64>
[ 4  6  0 ...  8 -1  4]
<bound method NDFrame.head of 0         91
1         58
2         59
3         66
4         58
          ..
281560    58
281561    89
281562    61
281563    79
281564    90
Name: age, Length: 281565, dtype: int64>
[11  8  0 ...  7  0  0]
<bound method NDFrame.head of 0         87
1         52
2         59
3         62
4         52
          ..
281560    54
281561    85
281562    53
281563    80
281564    86
Name: age, Length: 281565, dtype: int64>
